# K-means Clustering（K-均值聚类）
聚类属于无监督学习，是数据挖掘中常用算法。
常用于：
* 基于特征将用户群体分组
* 文档、日志数据聚类
* 风控欺诈行为识别
* 相似基因查找
* 生态学群体划分
* 图像识别检测

![](https://i.stack.imgur.com/cIDB3.png)

聚类算法中K-均值聚类是最简单的一种算法。
K-均值聚类试图将样本中的数据划分为K个不同的类簇（其中K是给定的输入参数）。它的目标是最小化数据样本到所属类簇中心（也称为质心）
我们可以形式化定义我们的目标为最小化:
\begin{equation}
\min \sum_{i=1}^k\sum_{{x}\in {C}_{i}}\lVert x - {u}_{i}\rVert^2
\end{equation}
换而言之，目标就是最小化样本与类簇质心的距离平方和

K-均值聚类的算法步骤可以描述为：
1. 随机分配K个类簇中心
2. 将样本数据分配到最近的质心
3. 重新计算各个类簇的中心，如果达到迭代次数或者**收敛**(通常可以定义为与上次类簇中心距离小于某个阈值)则结束算法，否则回到步骤2

我们通常选用欧式距离作为距离函数，欧式距离有一些很好的性质：
* d(i,j)>=0
* d(i,i)==0
* d(i,j)=d(j,i)
* d(i,j)<=d(i,k)+d(k,j)


![](http://konukoii.com/blog/wp-content/uploads/2017/01/RunyanKmeans.gif)

本篇介绍通过Spark实现K-均值聚类

## 数据使用说明
下面的程序采用的数据源自[IRIS数据集](http://archive.ics.uci.edu/ml/datasets/Iris)
IRIS共150个样本，分为3组，每组50条数据，每条数据包含4个属性:
1. 花萼长度
2. 花萼宽度P
3. 花瓣长度
4. 花瓣宽度

通过这四个属性来对鸢尾花数据集进行K-means聚类

## Python依赖
* NumPy
* Matplotlib

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_iris():
    return np.genfromtxt('iris.csv',
                       delimiter=',',
                       dtype=np.float)


In [ ]:
# 我们使用欧式距离的平方作为距离函数
def dist(a, b):
    c=a-b
    r=np.inner(c,c)
    return r

In [ ]:
def kmeans(dataset, k, eps=0.0001, max_iter_num=300):
    # 数据个数和特征维度
    count, features = dataset.shape

    # 随机初始化一组质心
    centers = dataset[np.random.randint(0, count - 1, size=k)]

    # belongs数组用来表示每条数据所属的质心编号
    belongs = np.zeros(count)

    iter_num = 1
    last_dist_sum = -1
    while iter_num <= max_iter_num:
        iter_num += 1

        # 计算所有鸢尾花到k个质心的距离，并更新belongs数组
        for i, iris in enumerate(dataset):
            dist_to_centers = np.zeros((k, 1))
            for j, center in enumerate(centers):
                dist_to_centers[j] = dist(center, iris)
            # 更新所在类簇
            belongs[i] = np.argmin(dist_to_centers)

        new_centers = np.zeros_like(centers)
        dist_sum = 0
        for i in range(len(centers)):
            # 捞取所有这轮迭代中被归到此质心的点的编号
            iris_in_cluster = [j for j in range(count) if belongs[j] == i]

            # 取平均值作为新的质心
            new_center = np.mean(dataset[iris_in_cluster], axis=0)
            new_centers[i] = new_center
            dist_sum += sum(dist(iris, new_center) for iris in iris_in_cluster)

        # 收敛则退出
        if last_dist_sum != -1 and abs(dist_sum - last_dist_sum) < eps:
            break
        last_dist_sum = dist_sum
        centers = new_centers
    return belongs, centers

In [ ]:
def plot_result(dataset, belongs, centers):
    colors = ['r', 'g', 'b']
    for i, iris in enumerate(dataset):
        plt.scatter(iris[0], iris[1], color=colors[int(belongs[i])])
    for point in centers:
        plt.scatter(point[0], point[1], color='k', s=80)

In [ ]:
dataset = load_iris()
# 花萼聚类
sepal = dataset[:, :2]
cluster, centers = kmeans(sepal, 3)
plot_result(sepal, cluster, centers)

In [ ]:
# 花瓣聚类
petal = dataset[:, 2:]
cluster, centers = kmeans(petal, 3)
plot_result(petal, cluster, centers)

## 使用Spark MLlib处理K-均值聚类
Spark的MLlib已经封装好了许多机器学习的算法，目前Spark中存在mllib与ml两个包，mllib主要基于RDD，而ml主要基于DataFrame。
下面的示例中我们使用mllib中的聚类算法对鸢尾花数据进行聚类，可以观察到与上面直接实现算法相同的结果。

In [ ]:
from pyspark.mllib.clustering import KMeans

def calculate_and_plot(dataset, k):
    model = KMeans.train(dataset, k)
    belongs = model.predict(dataset).collect()
    plot_result(dataset.collect(), belongs, model.clusterCenters)

In [ ]:
iris_rdd = sc.textFile('iris.csv')
# 花萼聚类
sepal = iris_rdd.map(lambda line: np.array([float(x) for x in line.split(',')][:2]))
calculate_and_plot(sepal, 3)

In [ ]:
# 花瓣聚类
petal = iris_rdd.map(lambda line: np.array([float(x) for x in line.split(',')][2:]))
calculate_and_plot(petal, 3)